In [1]:
!pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

from rapidfuzz import process, utils, fuzz
import re
import string

from pymystem3 import Mystem
from nltk.corpus import stopwords
from nltk import word_tokenize

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_excel('/content/drive/MyDrive/fuzzy_duplicates/test_5000.xlsx')
# df = pd.read_excel('test_5000.xlsx')
print(len(df))
df.dropna(subset='annotation', inplace=True)
print(len(df))

9900
9881


In [6]:
# предобработка текста
russian_stopwords_ss = stopwords.words("russian")
russian_stopwords_ss.extend(['который', "это", "такой", "сегодня", "еще", "точно", "спасибо", "весь", "все",
                             "каждый", "видимо", "также", "так", "сам", "пока", "ты", "быть", "просто", "почему",
                             "какой", "поэтому", "вообще", "например", "очень", "год", "вчера", "кто", "что",
                             "становиться", "говорить", "рассказывать", "делать", "сделать", "мочь", "видеть", "новый",
                             "место", "добрый", "день"])
russian_stopwords_smi = stopwords.words("russian")
russian_stopwords_smi.extend(["пресс", "служба", "который", "это", "этот", "сообщать", "также", "свой", "другой",
                              "поэтому", "число", "время", "российский", "страна", "сайт", "проект", "такой",
                              "год", "вопрос", "административый", "округ", "столица", "наш", "россия", "сегодня",
                              "каждый", "кроме", "должный"])

mystem = Mystem()

# Функция для предобработки строк текста
def preparation(texts, ss: bool):
    for i in range(len(texts)):
        # приводим к нижнему регистру
        texts[i] = str(texts[i]).lower()
        # оставляем только русские символы
        texts[i] = re.sub("[^а-яё]", " ", texts[i])
        # удаляем множественные пробелы
        texts[i] = re.sub(r"\s+", " ", texts[i], flags=re.I)
        # удаляем стоп слова и лемматизируем
        russian_stopwords = russian_stopwords_ss if ss else russian_stopwords_smi
        texts[i] = " ".join([token for token in word_tokenize(texts[i]) if token not in russian_stopwords and token != ' '])
    # лемматизируем
    texts = ' br '.join(texts)
    tokens = mystem.lemmatize(texts)
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    texts = " ".join(tokens)
    # После лемматизации, разбиваем тексты в список по спец-метке
    texts = texts.split(' br ')

    return texts

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [7]:
from transformers import AutoTokenizer, AutoModel
import torch
from concurrent.futures import ThreadPoolExecutor

def load_model_and_tokenizer(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModel.from_pretrained(model_name)
  if torch.cuda.is_available():
      model = model.to('cuda')  # Перемещение модели на GPU
  return [tokenizer, model]

def get_embedding(text, model, tokenizer):
    # Токенизация текста
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=64)
    # Получение векторов последнего скрытого слоя
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Возьмём среднее всех токенов в последнем слое и уберем лишнюю размерность
    embeddings = model_output.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy().tolist()  # Преобразуем тензор в NumPy массив для использования в функции cosine


# код для многопоточности и обработки по батчам ================================
def get_embeddings(texts, model, tokenizer, batch_size=32):
    # Разбиваем тексты на батчи
    batches = [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]
    # Создаём пул потоков
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda batch: get_batch_embedding(batch, model, tokenizer), batches))
    # Объединяем результаты из всех батчей
    return np.vstack(results).tolist()

def get_batch_embedding(text_batch, model, tokenizer):
    # Токенизация батча текстов
    encoded_input = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True, max_length=64)
    # Получение векторов последнего скрытого слоя
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Возьмём среднее всех токенов в последнем слое для каждого элемента в батче
    embeddings = model_output.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy()  # Преобразуем тензор в NumPy массив

In [9]:
from scipy.spatial.distance import cosine

# вычисление косинусной схожести векторов
def cosine_similarity(vec1, vec2):
    # Проверяем, равен ли один из векторов нулю
    if np.all(vec1 == 0) or np.all(vec2 == 0):
        if np.all(vec1 == 0) and np.all(vec2 == 0):
            return 1
        return 0  # Если один из векторов нулевой, сходство равно 0
    similarity = 1 - cosine(vec1, vec2)
    return similarity

# создание матрицы попарных сравнений
def create_combined_similarity_matrix(df1, df2=None):
    len_df1 = len(df1)

    # Определяем, сколько всего будет столбцов в матрице
    total_cols = len_df1 + (len(df2) if df2 is not None else 0)

    # Создаем пустую матрицу схожести
    similarity_matrix = np.zeros((len_df1, total_cols))

    # Заполнение матрицы схожести для внутренних сравнений df1
    for i in range(len_df1):
        for j in range(i+1, len_df1):
            sim = cosine_similarity(df1.iloc[i]['annotation_vectors'], df1.iloc[j]['annotation_vectors'])
            similarity_matrix[i, j] = sim
            similarity_matrix[j, i] = sim

    # Если df2 предоставлен, заполняем матрицу для сравнений между df1 и df2
    if df2 is not None:
        for i in range(len_df1):
            for j in range(len(df2)):
                sim = cosine_similarity(df1.iloc[i]['annotation_vectors'], df2.iloc[j]['annotation_vectors'])
                similarity_matrix[i, len_df1 + j] = sim

    return similarity_matrix

In [17]:
# добавление колонки dupl_id в new_df
def add_duplicate_ids(new_df, old_df=None, similarity_matrix=None, threshold=0.85):
    # Сброс индексов для корректной работы с индексами
    new_df = new_df.reset_index(drop=True)

    if old_df is not None:
        old_df = old_df.reset_index(drop=True)
        combined_df = pd.concat([new_df, old_df]).reset_index(drop=True)
    else:
        combined_df = new_df.copy()

    # Обновление new_df с новыми столбцами для ID дубликатов
    new_df['dupl_ids'] = None
    new_df['dupl_id_oldest'] = None

    # Если матрица схожести не предоставлена, создаем её
    if similarity_matrix is None:
        similarity_matrix = create_combined_similarity_matrix(new_df, old_df)

    # Обход каждой записи в new_df
    for i in new_df.index:
        high_similarity_indices = np.where(similarity_matrix[i] >= threshold)[0]
        similarities = similarity_matrix[i][high_similarity_indices]

        if high_similarity_indices.size > 0:
            # Получение дат всех схожих записей
            dates = combined_df.loc[high_similarity_indices, 'dt_object']

            # Нахождение самого старого документа
            oldest_index = dates.idxmin()

            # Присваивание ID самого старого документа и списка ID с уровнями схожести
            new_df.at[i, 'dupl_id_oldest'] = combined_df.at[oldest_index, 'id']
            new_df.at[i, 'dupl_ids'] = {combined_df.at[index, 'id']: round(sim, 4) for index, sim in zip(high_similarity_indices, similarities)}

    return new_df

In [18]:
def find_fuzzy_duplicates(new_df, old_df=None):
    # Задаем необходимые колонки и их функции генерации в словаре
    columns_to_add = {
        'preprocessed_annotation': lambda df: preparation(df['annotation'].tolist(), ss=True),
        'annotation_vectors': lambda df: df['preprocessed_annotation'].apply(lambda x: get_embedding(x, model, tokenizer)), # без многопоточности и батчей
        # 'annotation_vectors': lambda df: get_embeddings(df['preprocessed_annotation'].tolist(), model, tokenizer), # с многопоточностью и батчами
        'dupl_ids': lambda df: None,
        'dupl_id_oldest': lambda df: None
    }
    # Добавляем колонки, если они еще не существуют
    for column, func in columns_to_add.items():
        if column not in new_df.columns:
            new_df.loc[:, column] = func(new_df)
        if old_df is not None and column not in old_df.columns:
            old_df.loc[:, column] = func(old_df)

    similarity_matrix = create_combined_similarity_matrix(new_df, old_df)
    new_df = add_duplicate_ids(new_df, old_df, similarity_matrix)

    df = pd.concat([new_df, old_df]).reset_index(drop=True) if old_df is not None else new_df.reset_index(drop=True)
    return df

In [12]:
# удаляет дубликаты, оставляя только оригинал
def remove_fuzzy_duplicates(df):
    # Сортировка DataFrame по дате
    df = df.sort_values(['dt_object', 'id'])
    # Сортировка DataFrame по длине текста в annotation
    # df['annotation_length'] = df['annotation'].apply(len)
    # df = df.sort_values(['annotation_length', 'dt_object', 'id'], ascending=[False, True, True])
    # df = df.drop(columns=['annotation_length'])

    # Множество для отслеживания уже обработанных идентификаторов
    seen_ids = set()

    # Метки для удаления
    to_delete = []

    # Итерация по строкам DataFrame
    for index, row in df.iterrows():
        current_id = row['id']
        # Если текущий id уже встречался, помечаем строку для удаления
        if current_id in seen_ids:
            to_delete.append(index)
        else:
            # Проверяем, не является ли dupl_ids None
            if row['dupl_ids']:
                # Добавляем id из словаря dupl_ids в множество seen_ids
                seen_ids.update(row['dupl_ids'].keys())
            # Добавляем текущий id в множество seen_ids
            seen_ids.add(current_id)

    # Удаление дубликатов
    df = df.drop(to_delete)
    df = df.sort_index()

    return df

##Тестирование

In [13]:
model_name = 'cointegrated/rubert-tiny2'
# model_name = 'sentence-transformers/distiluse-base-multilingual-cased-v2'
# model_name = 'symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli'
# model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
# model_name = 'sentence-transformers/LaBSE'
# model_name = 'cointegrated/LaBSE-en-ru'
# model_name = 'DeepPavlov/rubert-base-cased'
# model_name = 'sberbank-ai/sbert_large_nlu_ru'
tokenizer, model = load_model_and_tokenizer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

In [19]:
new_df = df.head(50)
old_df = df.iloc[100:150]

In [20]:
%time result_df = find_fuzzy_duplicates(new_df, old_df)

<ipython-input-18-2b8733b0c6c9>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:, column] = func(new_df)
<ipython-input-18-2b8733b0c6c9>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_df.loc[:, column] = func(old_df)
<ipython-input-18-2b8733b0c6c9>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

CPU times: user 2.5 s, sys: 7.63 ms, total: 2.51 s
Wall time: 2.59 s


In [21]:
result_df.head()

,Unnamed: 0,id,is_sent,system,source_type,message_id,report_id,report_name,link,title,...,time_base,blog_link,is_ppb,subscribers,author,author_link,preprocessed_annotation,annotation_vectors,dupl_ids,dupl_id_oldest
0,100,5939510,False,Медиалогия,Соцсети,100876048228,3480354,Департамент ПиООС,https://telegram.me/stranaru/58686,NaN,...,2023-10-28 22:24:11,https://t.me/stranaru,False,0,СТРАНА RU,https://t.me/stranaru,необъявленный вырубка дерево битцевский лес ох...,"[0.08174809068441391, -0.4723580479621887, 1.0...","{5939513: 1.0, 5939515: 1.0, 5939631: 1.0, 593...",5939515
1,101,5939511,False,Медиалогия,Соцсети,100876014412,3214233,Подведомственные предприятия и органы исп. вла...,https://twitter.com/chrergk/status/17183453813...,NaN,...,2023-10-28 22:24:36,https://twitter.com/chrergk,False,1,cha-,https://twitter.com/chrergk,подарочек альбедо кли авто привет альбедо альб...,"[-0.2000746726989746, -0.33551621437072754, 0....",None,None
2,102,5939512,False,Медиалогия,Соцсети,100876042262,3214233,Подведомственные предприятия и органы исп. вла...,https://vk.com/wall-221568689_429,NaN,...,2023-10-28 22:24:37,https://vk.com/club221568689,False,40,Приют «Пушистый друг». Сектор Е.,https://vk.com/club221568689,выгул погода получаться продолжительный посеща...,"[0.6122724413871765, -0.3316040635108948, 0.62...",{5939519: 1.0},5939519
3,103,5939513,False,Медиалогия,Соцсети,100875349651,3214233,Подведомственные предприятия и органы исп. вла...,https://telegram.me/shto_tam_v_rossii/75263,NaN,...,2023-10-28 22:24:38,https://t.me/shto_tam_v_rossii,False,13,Что там в России?,https://t.me/shto_tam_v_rossii,необъявленный вырубка дерево битцевский лес ох...,"[0.08174809068441391, -0.4723580479621887, 1.0...","{5939510: 1.0, 5939515: 1.0, 5939631: 1.0, 593...",5939510
4,104,5939514,False,Медиалогия,Соцсети,100875248741,3214233,Подведомственные предприятия и органы исп. вла...,https://telegram.me/polit_birga/58761,NaN,...,2023-10-28 22:24:38,https://t.me/polit_birga,False,0,Политическая биржа,https://t.me/polit_birga,москвич обнаруживать метровый змея свой кварти...,"[-0.3871988356113434, -0.43797025084495544, 1....","{5939530: 1.0, 5939531: 1.0, 5939534: 1.0, 593...",5939530


In [22]:
print(result_df[result_df['id'] == 5939510]['annotation'].tolist()[0])
print(result_df[result_df['id'] == 5939515]['annotation'].tolist()[0])

Необъявленная Вырубка Деревьев в Битцевском Лесу:... Охрана Природы в Действии Автор: [Ваше имя] В недавнем инциденте, который произошел в районе Анино в Битцевском лесу, столичные общественные инспекторы от <b>ДПиООС</b>, Росприроднадзора и КПЭБ быстро отреагировали на сигнал о незаконной порубке деревьев.... Неизвестные лица активно вырубали деревья, и местные жители беспокойно обратились за помощью к экологическим организациям.... По...
Необъявленная Вырубка Деревьев в Битцевском Лесу:... Охрана Природы в Действии Автор: [Ваше имя] В недавнем инциденте, который произошел в районе Анино в Битцевском лесу, столичные общественные инспекторы от <b>ДПиООС</b>, Росприроднадзора и КПЭБ быстро отреагировали на сигнал о незаконной порубке деревьев.... Неизвестные лица активно вырубали деревья, и местные жители беспокойно обратились за помощью к экологическим организациям.... По...


In [23]:
no_dupl_df = remove_fuzzy_duplicates(result_df).reset_index(drop=True, inplace=True)

In [33]:
no_dupl_df

,Unnamed: 0,id,is_sent,system,source_type,message_id,report_id,report_name,link,title,...,time_base,blog_link,is_ppb,subscribers,author,author_link,preprocessed_annotation,annotation_vectors,dupl_ids,dupl_id_oldest
0,100,5939510,False,Медиалогия,Соцсети,100876048228,3480354,Департамент ПиООС,https://telegram.me/stranaru/58686,NaN,...,2023-10-28 22:24:11,https://t.me/stranaru,False,0,СТРАНА RU,https://t.me/stranaru,необъявленный вырубка дерево битцевский лес ох...,"[0.08174809068441391, -0.4723580479621887, 1.0...","{5939513: 1.0, 5939515: 1.0, 5939631: 1.0, 593...",5939515
1,101,5939511,False,Медиалогия,Соцсети,100876014412,3214233,Подведомственные предприятия и органы исп. вла...,https://twitter.com/chrergk/status/17183453813...,NaN,...,2023-10-28 22:24:36,https://twitter.com/chrergk,False,1,cha-,https://twitter.com/chrergk,подарочек альбедо кли авто привет альбедо альб...,"[-0.2000746726989746, -0.33551621437072754, 0....",None,None
2,102,5939512,False,Медиалогия,Соцсети,100876042262,3214233,Подведомственные предприятия и органы исп. вла...,https://vk.com/wall-221568689_429,NaN,...,2023-10-28 22:24:37,https://vk.com/club221568689,False,40,Приют «Пушистый друг». Сектор Е.,https://vk.com/club221568689,выгул погода получаться продолжительный посеща...,"[0.6122724413871765, -0.3316040635108948, 0.62...",{5939519: 1.0},5939519
3,105,5945512,False,Brand Analytics,Соцсети,1000031752,12454959,Районные сообщества ЗелАО,https://vk.com/wall-30252262_167797?reply=1679...,NaN,...,2023-10-29 23:30:15,NaN,False,50410,Сергей Попов,https://vk.com/id35514692,алексей спартаковский цвет белый красный белый...,"[1.0179803371429443, -0.4119804799556732, 0.35...",None,None
4,108,5939517,False,Brand Analytics,Соцсети,1000087342,12458870,Районные сообщества ВАО 2,https://vk.com/wall-136569697_78407?reply=78423,NaN,...,2023-10-28 22:25:55,NaN,False,27584,Дмитрий Никитин,https://vk.com/id66476822,думать лишь гавно набросать портить здоровье н...,"[-0.11373618245124817, -0.3847554922103882, 0....",{5939524: 1.0},5939524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,247,5939652,False,Brand Analytics,Соцсети,1000310898,12450327,ЧП Москва,https://telegram.me/v_msk_escort/1390204,NaN,...,2023-10-28 22:59:51,https://t.me/rita_belorus,False,1571,NaN,NaN,мужчина хотеть потушить свой пожар спешить пря...,"[-0.19950294494628906, -0.44890573620796204, 0...",None,None
57,248,5939653,False,Brand Analytics,Соцсети,1000310899,12450327,ЧП Москва,https://telegram.me/ZnakomstvaMskChat/2578355,NaN,...,2023-10-28 22:59:52,https://t.me/rita_belorus,False,718,NaN,NaN,мужчина хотеть потушить свой пожар спешить пря...,"[-0.19950294494628906, -0.44890573620796204, 0...",None,None
58,249,5939654,False,Brand Analytics,Соцсети,1000310902,12450327,ЧП Москва,https://telegram.me/girlrab/20050815,NaN,...,2023-10-28 22:59:52,https://t.me/rita_belorus,False,2268,NaN,NaN,мужчина хотеть потушить свой пожар спешить пря...,"[-0.19950294494628906, -0.44890573620796204, 0...",None,None
59,250,5939655,False,Brand Analytics,Соцсети,1000310897,12450327,ЧП Москва,https://telegram.me/soderganci/106553,NaN,...,2023-10-28 22:59:52,https://t.me/rita_belorus,False,1826,NaN,NaN,мужчина хотеть потушить свой пожар спешить пря...,"[-0.19950294494628906, -0.44890573620796204, 0...",None,None
